In [3]:
import pandas as pd
import nltk

In [4]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [5]:
sia = SentimentIntensityAnalyzer() 

In [6]:
import pyodbc

In [26]:
connection1 = (

    "Driver={SQL Server};"
    "Server=MUKTA\SQLEXPRESS;"  #I didn't put one extra slash, /- escape character.
    "Database=PortfolioProject_MarketingAnalytics;"
    "Trusted_Connection=yes;"

)

conn = pyodbc.connect(connection)

<>:4: SyntaxWarning: invalid escape sequence '\S'
<>:4: SyntaxWarning: invalid escape sequence '\S'
C:\Users\jainm\AppData\Local\Temp\ipykernel_31688\1356931513.py:4: SyntaxWarning: invalid escape sequence '\S'
  "Server=MUKTA\SQLEXPRESS;"  #I didn't put one extra slash, /- escape character.


In [8]:
connection = (

    "Driver={SQL Server};"
    "Server=MUKTA\\SQLEXPRESS;"  #I need to put one extra slash
    "Database=PortfolioProject_MarketingAnalytics;"
    "Trusted_Connection=yes;"

)

#This is a mulitiline string

conn = pyodbc.connect(connection)

In [9]:
query = "SELECT ReviewID, CustomerID, ProductID, ReviewDate, Rating,REPLACE(ReviewText, '  ', ' ') as ReviewText FROM customer_reviews"

In [10]:
df = pd.read_sql(query,conn)

C:\Users\jainm\AppData\Local\Temp\ipykernel_31688\785092515.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query,conn)


In [11]:
df

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText
0,1,77,18,2023-12-23,3,"Average experience, nothing special."
1,2,80,19,2024-12-25,5,The quality is top-notch.
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper."
4,5,64,2,2023-07-16,3,"Average experience, nothing special."
...,...,...,...,...,...,...
1358,1359,28,4,2023-05-25,3,Not worth the money.
1359,1360,58,12,2023-11-13,2,"Average experience, nothing special."
1360,1361,96,15,2023-03-07,5,Customer support was very helpful.
1361,1362,99,2,2025-12-03,1,Product did not meet my expectations.


In [12]:
sia.polarity_scores(df['ReviewText'][0])

{'neg': 0.429, 'neu': 0.571, 'pos': 0.0, 'compound': -0.3089}

In [13]:
b = df['ReviewText'][0]

In [14]:
b

'Average experience, nothing special.'

In [15]:
type(b)

str

In [16]:
sia.polarity_scores(b)['compound']

-0.3089

In [17]:
df["Review_Scores"] = df['ReviewText'].apply(lambda x : sia.polarity_scores(x)['compound'])

In [18]:
df

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText,Review_Scores
0,1,77,18,2023-12-23,3,"Average experience, nothing special.",-0.3089
1,2,80,19,2024-12-25,5,The quality is top-notch.,0.0000
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.,0.0000
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper.",0.2382
4,5,64,2,2023-07-16,3,"Average experience, nothing special.",-0.3089
...,...,...,...,...,...,...,...
1358,1359,28,4,2023-05-25,3,Not worth the money.,-0.1695
1359,1360,58,12,2023-11-13,2,"Average experience, nothing special.",-0.3089
1360,1361,96,15,2023-03-07,5,Customer support was very helpful.,0.6997
1361,1362,99,2,2025-12-03,1,Product did not meet my expectations.,0.0000


In [53]:
#Let's bring out a result 
'''
score >0.05 and rating >=4  positive
score >0.05 and rating ==3 mixed positive 
score >0.05 and rating<3  mixed negative 
score < -0.05 and rating <3 negative 
score < -0.05 and rating ==3 mixed negative 
score < -0.05 and rating >3 mixed positive 
score anything other than this, see rating..rating>=4 positive , rating==3 neutral and rating <3 negative 
'''

'\nscore >0.05 and rating >=4  positive\nscore >0.05 and rating ==3 mixed positive \nscore >0.05 and rating<3  mixed negative \nscore <0.05 and rating <3 negative \nscore <0.05 and rating ==3 mixed negative \nscore <0.05 and rating >3 mixed positive \nscore anything other than this, see rating..rating>=4 positive , rating==3 neutral and rating <3 negative \n'

In [19]:
def category_sentiment(score, rating):
    if score >= 0.05:
        if rating >3:
            return 'Positive'
        elif rating ==3:
            return 'Mixed Positive'
        else:
            return 'Mixed Negative'
        
    elif score <= -0.05 :
        if rating >3:
            return 'Mixed Positive'
        elif rating ==3:
            return 'Mixed negative'
        else:
            return 'Negative'
        
    else:
        if rating >3:
            return 'Positive'
        elif rating ==3:
            return 'Neutral'
        else:
            return 'Negative'

In [20]:
df['SentimentCategory'] = df.apply(lambda x : category_sentiment(x["Review_Scores"], x["Rating"]), axis =1)

In [21]:
df

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText,Review_Scores,SentimentCategory
0,1,77,18,2023-12-23,3,"Average experience, nothing special.",-0.3089,Mixed negative
1,2,80,19,2024-12-25,5,The quality is top-notch.,0.0000,Positive
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.,0.0000,Positive
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper.",0.2382,Mixed Positive
4,5,64,2,2023-07-16,3,"Average experience, nothing special.",-0.3089,Mixed negative
...,...,...,...,...,...,...,...,...
1358,1359,28,4,2023-05-25,3,Not worth the money.,-0.1695,Mixed negative
1359,1360,58,12,2023-11-13,2,"Average experience, nothing special.",-0.3089,Negative
1360,1361,96,15,2023-03-07,5,Customer support was very helpful.,0.6997,Positive
1361,1362,99,2,2025-12-03,1,Product did not meet my expectations.,0.0000,Negative


In [22]:
def sentiment_bucket(score):
    # Positive
    if score>= 0.5:
        return '[0.5 , 1]'  
    # Negative
    elif score <= -0.5:
        return '[-1, -0.5]'
    # Mildly Negative
    elif score > -0.5 and score < 0:
        return '(-0.5,0)'
    # Mildly Positive
    else:
        return '[0, 0.5)'
    

     

In [23]:
df['SentimentBucket'] = df['Review_Scores'].apply(lambda x : sentiment_bucket(x) )

In [24]:
df

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText,Review_Scores,SentimentCategory,SentimentBucket
0,1,77,18,2023-12-23,3,"Average experience, nothing special.",-0.3089,Mixed negative,"(-0.5,0)"
1,2,80,19,2024-12-25,5,The quality is top-notch.,0.0000,Positive,"[0, 0.5)"
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.,0.0000,Positive,"[0, 0.5)"
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper.",0.2382,Mixed Positive,"[0, 0.5)"
4,5,64,2,2023-07-16,3,"Average experience, nothing special.",-0.3089,Mixed negative,"(-0.5,0)"
...,...,...,...,...,...,...,...,...,...
1358,1359,28,4,2023-05-25,3,Not worth the money.,-0.1695,Mixed negative,"(-0.5,0)"
1359,1360,58,12,2023-11-13,2,"Average experience, nothing special.",-0.3089,Negative,"(-0.5,0)"
1360,1361,96,15,2023-03-07,5,Customer support was very helpful.,0.6997,Positive,"[0.5 , 1]"
1361,1362,99,2,2025-12-03,1,Product did not meet my expectations.,0.0000,Negative,"[0, 0.5)"


In [25]:
df.to_csv('fact_customer_reviews_with_sentiment.csv' , index = False)